In [159]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import cv2
import copy

In [160]:
batchsize = 32*4
seed = 0

In [161]:
present=r'../input/brain-mri-images-for-brain-tumor-detection/brain-mri-images-for-brain-tumor-detection/yes/*.*'
notpresent=r'../input/brain-mri-images-for-brain-tumor-detection/brain-mri-images-for-brain-tumor-detection/no/*.*'

In [162]:
dataset=[]

y_true=[]

from skimage.transform import resize

pngs_p=glob(present)

pngs_n=glob(notpresent)

for i in range(len(pngs_p)):
    
    img=cv2.imread(pngs_p[i],0)
    img=resize(img,(64,64))
    dataset.append(img)
    y_true.append(int(1))
    
for i in range(len(pngs_n)):
    
    img=cv2.imread(pngs_n[i],0)
    img=resize(img,(64,64))
    dataset.append(img)
    y_true.append(int(0))

In [177]:
len(dataset)

253

In [176]:
len(y_true)

253

In [164]:
def CNN_neural():
    # Indicates that our model is built using Sequential layers
    model=Sequential()
    
    # First, we add multiples convolution layers to find patterns
    model.add(Conv2D(filters=32,kernel_size=(3,3),strides=1,activation='relu',input_shape=(64,64,1)))
    # Next, we add Pooling layer to reduce the size and find the occurence of feature in the convolution set
    model.add(MaxPooling2D(pool_size=(2,2)))
    # Scales the outputs of previous layers
    model.add(BatchNormalization(axis=-1))
    
    # We repeat the same to create a slightly complex model
    model.add(Conv2D(filters=32,kernel_size=(3,3),strides=1,activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization(axis=-1))
    
    model.add(Conv2D(filters=32,kernel_size=(3,3),strides=1,activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization(axis=-1))
    
    # To prevent overfitting
    model.add(Dropout(rate=0.25))
    
    # Now we begin the construction of ANN with the above network output as input
    model.add(Flatten())
    model.add(Dense(256,activation='relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(Dropout(rate=0.25))
    model.add(Dense(1,activation='softmax'))
    
    return model

In [165]:
model=CNN_neural()

In [166]:
model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [167]:
X_train,X_test= train_test_split(dataset,test_size=0.1, shuffle=False,random_state=0)

In [171]:
y_train,y_test= train_test_split(y_true,test_size=0.1,shuffle=False,random_state=0)

In [ ]:
'''X_train=np.array(X_train)
y_train=np.array(y_train)

X_test=np.array(X_test)
y_test=np.array(y_test)'''


In [172]:
X_train=np.resize(X_train, (-1, 64,64))

In [173]:
X_train=np.expand_dims(X_train,axis=3)

In [174]:
X_train.shape

(226, 64, 64, 1)

In [175]:
model.fit(x=X_train, y=y_train, batch_size=None,epochs=100, verbose=1, callbacks=None, validation_split=0.1,validation_data=None, shuffle=True, steps_per_epoch=202)


ValueError: Input arrays should have the same number of samples as target arrays. Found 226 input samples and 227 target samples.

In [ ]:
plt.plot(output.history['acc'])
plt.plot(output.history['val_acc'])
plt.title('classifier  based accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(output.history['loss'])
plt.plot(output.history['val_loss'])
plt.title('classifier based loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()